In [1]:
import torch
import random

from transformers import AutoTokenizer, AutoModelWithLMHead

In [2]:
tokenizer = AutoTokenizer.from_pretrained("openai-gpt")

model = AutoModelWithLMHead.from_pretrained("openai-gpt")


In [3]:
def generate_text(input_text: str, tokens_to_generate: int):
    text_generated = torch.tensor([tokenizer.encode(input_text)], dtype=torch.long)
    with torch.no_grad():
        for _ in range(tokens_to_generate):
            predictions = model(text_generated)
            next_token_logits = predictions[0][:, -1, :]

            next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(-1)
            
            text_generated = torch.cat((text_generated, next_token_id), dim=1)
    result = tokenizer.decode(text_generated.squeeze().tolist())
    return result

In [4]:
text = "The Hamilton is a work of"

In [5]:
text_encoded = torch.tensor([tokenizer.encode(text)], dtype=torch.long)

In [6]:
model.eval()
with torch.no_grad():
    predictions = model(text_encoded)

In [7]:
print(generate_text(text, 100))

the hamilton is a work of art. " 
 " i'm sure it is. " 
 " i'm sure it is. " 
 " i'm sure it is. " 
 " i'm sure it is. " 
 " i'm sure it is. " 
 " i'm sure it is. " 
 " i'm sure it is. " 
 " i'm sure it is. " 
 " i'm sure it is. " 
 " i'm sure it is. " 
 " i'm sure it is


In [56]:
def gen_topK_text(input_text: str, tokens_to_generate: int, k: int):
    text_generated = torch.tensor([tokenizer.encode(input_text)], dtype=torch.long)
    with torch.no_grad():
        for _ in range(tokens_to_generate):
            predictions = model(text_generated)
            
            next_token_logits = predictions[0][:, -1, :]
            next_token_id = torch.LongTensor(random.choices(next_token_logits.topk(k, dim=-1).indices.tolist()[0], torch.nn.functional.softmax(next_token_logits).topk(k, dim=-1).values.tolist()[0])).unsqueeze(-1)            
            text_generated = torch.cat((text_generated, next_token_id), dim=1)
    result = tokenizer.decode(text_generated.squeeze().tolist())
    return result

In [61]:
print(gen_topK_text(text, 100, 10))

/Users/macbook/PycharmProjects/nlp-course/venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


the hamilton is a work of art... " 
 " i know, " replied janet. 
 " you must be impressed. " 
 " i am. it's a wonderful place and i can't wait to see the others, " admitted janet. 
 " i hope they aren't too tired from yesterday's activities... " 
 janet shook her head. " i hope not! and if the others didn't make it, i think they will be more interested in what i've learned tomorrow. " 
 chapter 14 
 a short


In [68]:
print(gen_topK_text(text, 100, 3))


/Users/macbook/PycharmProjects/nlp-course/venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


the hamilton is a work of art. " 
 " it's a beautiful place, " i agreed. 
 " i've been here before. " she smiled at me and i felt a pang of jealousy. i wasn't sure why, but i wanted to see her again, to be near her, even if she hadn't been there before. 
 " i'm glad you're here, " i said softly, my eyes on the painting of the two of us. 
 " me too. " 
 " i'm glad


In [69]:
def gen_topK_text_penalized(input_text: str, tokens_to_generate: int, k: int, penalty: float):
    text_generated = torch.tensor([tokenizer.encode(input_text)], dtype=torch.long)
    with torch.no_grad():
        for _ in range(tokens_to_generate):
            predictions = model(text_generated)
            
            next_token_logits = predictions[0][:, -1, :]
            
            #applying penalty (>1 to cut repetition) to logits that are already in text_generated
            #we add (penalty-1)% to logits < 0 and substract (penalty-1)% if logits > 0

            if penalty != 1:
              for i in range(len(next_token_logits)):
                if i in text_generated[0]:
                  if next_token_logits[0][i]>=0:
                    next_token_logits[0][i] /= penalty  
                  else:
                    next_token_logits[0][i] *= penalty

            next_token_id = torch.LongTensor(random.choices(next_token_logits.topk(k, dim=-1).indices.tolist()[0], torch.nn.functional.softmax(next_token_logits).topk(k, dim=-1).values.tolist()[0])).unsqueeze(-1)   
            
            text_generated = torch.cat((text_generated, next_token_id), dim=1)
    result = tokenizer.decode(text_generated.squeeze().tolist())
    return result


In [78]:
print(gen_topK_text_penalized(text, 200, 3, 1.7))


/Users/macbook/PycharmProjects/nlp-course/venv/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


the hamilton is a work of art, " he said. 
 " i know, " i said, " but i'm not a fan. " 
 " i know. " 
 " you're a good guy, " i said, " but i'm not a fan. i'm not a fan of anything. " 
 " i know, " he said again. " but i'm a fan of you. " 
 " i'm not a fan of you, " i repeated. 
 " i know. " 
 " you don't know. " 
 " i know, " he agreed. 
 i sighed. " okay, then, i'm not a fan of you. " 
 " good, " he said. " because i'm going to kiss you. " 
 i was still breathing hard. i couldn't help but notice that he hadn't taken his eyes off me, and i wasn't sure whether that was because he wanted to kiss or because of my reaction. 
 " you're a good


In [ ]:
def gen_topK_text_penalized(input_text: str, tokens_to_generate: int, k: int, penalty: float):
    text_generated = torch.tensor([tokenizer.encode(input_text)], dtype=torch.long)
    with torch.no_grad():
        for _ in range(tokens_to_generate):
            predictions = model(text_generated)
            
            next_token_logits = predictions[0][:, -1, :]
            
            #applying penalty (>1 to cut repetition) to logits that are already in text_generated
            #we add (penalty-1)% to logits < 0 and substract (penalty-1)% if logits > 0

            if penalty != 1:
              for i in range(len(next_token_logits)):
                if i in text_generated[0]:
                  if next_token_logits[0][i]>=0:
                    next_token_logits[0][i] /= penalty  
                  else:
                    next_token_logits[0][i] *= penalty

            #next_token_id = torch.LongTensor(random.choices(next_token_logits.topk(k, dim=-1).indices.tolist()[0], torch.nn.functional.softmax(next_token_logits).topk(k, dim=-1).values.tolist()[0])).unsqueeze(-1)   
            
            text_generated = torch.cat((text_generated, next_token_id), dim=1)
    result = tokenizer.decode(text_generated.squeeze().tolist())
    return result

